In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(return_messages=True)
def load_history(_):
    return memory.load_memory_variables({})["history"]

loader = UnstructuredFileLoader("./files/doc2.txt")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

cache_dir = LocalFileStore("./.cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

def stuff_docs(documents):
    return "\n\n".join(d.page_content for d in documents)

context_chain = retriever | RunnableLambda(stuff_docs)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer ONLY using the context below. "
            "If you don't know, say you don't know.\n\n"
            "----- CONTEXT -----\n{context}\n--------------------"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": context_chain,
        "history": RunnableLambda(load_history),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

def ask(question: str):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)
    return result.content

ask("Aaronson 은 유죄인가요?")
ask("그가 테이블에 어떤 메시지를 썼나요?")
ask("Julia 는 누구인가요?")


제가 알기로는 Aaronson은 유죄로 판결받았습니다.
그가 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.
Julia는 Winston과 함께 이야기 중에 등장하는 여성 캐릭터입니다.


'Julia는 Winston과 함께 이야기 중에 등장하는 여성 캐릭터입니다.'